# Recommendor System

Importing all required libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from random import randint

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00   1.03 MB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00   1.57 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   1.17 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  29.08 MB/s
vincent-0.4.4- 100% |###################

### Defining Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'DCL1APZZPRAHBRPEWVJ1ZWMEQ0VDJI3Q2E3VVPHTNZRCARYX' # your Foursquare ID
CLIENT_SECRET = 'PBFBXL1IN4RPBMMNY2IBJE5M55MWBCR2KT253GBC5TF4P2PF' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

credentails:
CLIENT_ID: DCL1APZZPRAHBRPEWVJ1ZWMEQ0VDJI3Q2E3VVPHTNZRCARYX
CLIENT_SECRET:PBFBXL1IN4RPBMMNY2IBJE5M55MWBCR2KT253GBC5TF4P2PF


Converting address to Lat and Long coordinates

In [3]:
address = "Noida Sector 62, UP"

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print(latitude,longitude)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


28.6169843 77.3736012


## 1. Search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

In [23]:
search_query = 'Restaurant'
# radius = 2000
# LIMIT = 100
print(search_query + ' .... OK!',end='\n\n')

# url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query)

results = requests.get(url).json()


### Visualizing names of all restaraunts
results['response']['venues'][0]['name']

Restaurant .... OK!



"Dev's  Restaurant & Bar"

In [24]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1c4941735', 'shortName...",False,52978e9811d2ca6acc6743b3,"Sector 62,Noida",IN,NaN,India,NaN,350,"[Sector 62,Noida, Uttar Pradesh, India]","[{'lng': 77.37140893936157, 'lat': 28.61449545...",28.614495,77.371409,NaN,Uttar Pradesh,Dev's Restaurant & Bar,v-1542631358,NaN
1,"[{'id': '4bf58dd8d48988d10f941735', 'shortName...",False,521ccaea11d2a569f484368c,NaN,IN,NaN,India,NaN,669,[India],"[{'lng': 77.36777960910399, 'lat': 28.61381660...",28.613817,77.367780,NaN,NaN,shakuntalam sweets namkeen & restaurant,v-1542631358,NaN
2,"[{'id': '4bf58dd8d48988d10f941735', 'shortName...",False,4e9710167ee65b1674cd21e8,H1A/31,IN,Noida,India,COMMERCIAL MARKET SEC-63,1141,"[H1A/31 (COMMERCIAL MARKET SEC-63), Noida, Utt...","[{'lng': 77.3753215547607, 'lat': 28.627122524...",28.627123,77.375322,NaN,Uttar Pradesh,The Yellow Chilli Restaurant,v-1542631358,NaN
3,"[{'id': '4bf58dd8d48988d10f941735', 'shortName...",False,52dfa8f211d254acf595bd1c,"B-1, 58/59, Central Market, Sec-50, Noida-201...",IN,Noida,India,at Sector 50,1328,"[B-1, 58/59, Central Market, Sec-50, Noida-20...","[{'lng': 77.36127020000004, 'lat': 28.6119524,...",28.611952,77.361270,201301,Uttar Pradesh,Khidmat Restaurant,v-1542631358,NaN
4,"[{'id': '4bf58dd8d48988d142941735', 'shortName...",False,4ece290df5b9832ad7a6d184,NaN,IN,NaN,India,NaN,1832,[India],"[{'lng': 77.354904, 'lat': 28.61572, 'label': ...",28.615720,77.354904,NaN,NaN,Bonanza Restaurant Vaishali sec 4,v-1542631358,NaN
5,"[{'id': '4bf58dd8d48988d147941735', 'shortName...",False,56d44a12cd10b9cf62f236ff,"Shop No. 1,Vaibav Khand,Indirapuram",IN,Delhi-NCR,India,Krishna Apra Sapphire Plaza,2356,"[Shop No. 1,Vaibav Khand,Indirapuram (Krishna ...","[{'lng': 77.365671, 'lat': 28.636972, 'label':...",28.636972,77.365671,201301,Delhi,1 The Restaurant,v-1542631358,NaN
6,"[{'id': '4bf58dd8d48988d1c3941735', 'shortName...",False,4b644892f964a52041a82ae3,3/48 MALCHA MARG SHOPPING CENTRE CHANKYAPURI,IN,New Delhi,India,NaN,18373,"[3/48 MALCHA MARG SHOPPING CENTRE CHANKYAPURI,...","[{'lng': 77.1863227351158, 'lat': 28.602451108...",28.602451,77.186323,110021,Delhi,FEZ Restaurant,v-1542631358,86873884
7,"[{'id': '4bf58dd8d48988d1c4941735', 'shortName...",False,5297919411d2ca6acc682ff0,"Shop No. 12A, Lotus Plaza,Vaibhav Khand, Near ...",IN,Ghāziābād,India,NaN,2371,"[Shop No. 12A, Lotus Plaza,Vaibhav Khand, Near...","[{'lng': 77.36394166946411, 'lat': 28.63653228...",28.636532,77.363942,NaN,Uttar Pradesh,Galords Restaurant,v-1542631358,NaN
8,"[{'id': '4bf58dd8d48988d10f941735', 'shortName...",False,4e7dbf44f9f46ece9c8e489c,Bapu Nature Cure Hospital & Yogashram,IN,New Delhi,India,Mayur Vihar Phase - 1,8399,[Bapu Nature Cure Hospital & Yogashram (Mayur ...,"[{'lng': 77.28783130645752, 'lat': 28.61206547...",28.612065,77.287831,110091,Delhi,Green Leaf Restaurant,v-1542631358,45370421
9,"[{'id': '4d4b7105d754a06374d81259', 'shortName...",False,4e6a4cce52b1c1205fbf350b,"28, 29, 31, Express Garden Market, 6- Vaibhav ...",IN,"Indrapuram, Ghaziabad",India,Near Shipra Mall,2411,"[28, 29, 31, Express Garden Market, 6- Vaibhav...","[{'lng': 77.3690915107727, 'lat': 28.638283730...",28.638284,77.369092,NaN,NaN,Haveli Restaurant,v-1542631358,NaN


In [25]:
# keeping only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

    
# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Dev's Restaurant & Bar,Restaurant,"Sector 62,Noida",IN,NaN,India,NaN,350,"[Sector 62,Noida, Uttar Pradesh, India]","[{'lng': 77.37140893936157, 'lat': 28.61449545...",28.614495,77.371409,NaN,Uttar Pradesh,52978e9811d2ca6acc6743b3
1,shakuntalam sweets namkeen & restaurant,Indian Restaurant,NaN,IN,NaN,India,NaN,669,[India],"[{'lng': 77.36777960910399, 'lat': 28.61381660...",28.613817,77.367780,NaN,NaN,521ccaea11d2a569f484368c
2,The Yellow Chilli Restaurant,Indian Restaurant,H1A/31,IN,Noida,India,COMMERCIAL MARKET SEC-63,1141,"[H1A/31 (COMMERCIAL MARKET SEC-63), Noida, Utt...","[{'lng': 77.3753215547607, 'lat': 28.627122524...",28.627123,77.375322,NaN,Uttar Pradesh,4e9710167ee65b1674cd21e8
3,Khidmat Restaurant,Indian Restaurant,"B-1, 58/59, Central Market, Sec-50, Noida-201...",IN,Noida,India,at Sector 50,1328,"[B-1, 58/59, Central Market, Sec-50, Noida-20...","[{'lng': 77.36127020000004, 'lat': 28.6119524,...",28.611952,77.361270,201301,Uttar Pradesh,52dfa8f211d254acf595bd1c
4,Bonanza Restaurant Vaishali sec 4,Asian Restaurant,NaN,IN,NaN,India,NaN,1832,[India],"[{'lng': 77.354904, 'lat': 28.61572, 'label': ...",28.615720,77.354904,NaN,NaN,4ece290df5b9832ad7a6d184
5,1 The Restaurant,Diner,"Shop No. 1,Vaibav Khand,Indirapuram",IN,Delhi-NCR,India,Krishna Apra Sapphire Plaza,2356,"[Shop No. 1,Vaibav Khand,Indirapuram (Krishna ...","[{'lng': 77.365671, 'lat': 28.636972, 'label':...",28.636972,77.365671,201301,Delhi,56d44a12cd10b9cf62f236ff
6,FEZ Restaurant,Moroccan Restaurant,3/48 MALCHA MARG SHOPPING CENTRE CHANKYAPURI,IN,New Delhi,India,NaN,18373,"[3/48 MALCHA MARG SHOPPING CENTRE CHANKYAPURI,...","[{'lng': 77.1863227351158, 'lat': 28.602451108...",28.602451,77.186323,110021,Delhi,4b644892f964a52041a82ae3
7,Galords Restaurant,Restaurant,"Shop No. 12A, Lotus Plaza,Vaibhav Khand, Near ...",IN,Ghāziābād,India,NaN,2371,"[Shop No. 12A, Lotus Plaza,Vaibhav Khand, Near...","[{'lng': 77.36394166946411, 'lat': 28.63653228...",28.636532,77.363942,NaN,Uttar Pradesh,5297919411d2ca6acc682ff0
8,Green Leaf Restaurant,Indian Restaurant,Bapu Nature Cure Hospital & Yogashram,IN,New Delhi,India,Mayur Vihar Phase - 1,8399,[Bapu Nature Cure Hospital & Yogashram (Mayur ...,"[{'lng': 77.28783130645752, 'lat': 28.61206547...",28.612065,77.287831,110091,Delhi,4e7dbf44f9f46ece9c8e489c
9,Haveli Restaurant,Food,"28, 29, 31, Express Garden Market, 6- Vaibhav ...",IN,"Indrapuram, Ghaziabad",India,Near Shipra Mall,2411,"[28, 29, 31, Express Garden Market, 6- Vaibhav...","[{'lng': 77.3690915107727, 'lat': 28.638283730...",28.638284,77.369092,NaN,NaN,4e6a4cce52b1c1205fbf350b


## Visualising nearby restaraunts

In [81]:
# create map of Noida using latitude and longitude values
map_Noida = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(dataframe_filtered['lat'], dataframe_filtered['lng'], dataframe_filtered['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Noida)  
    
map_Noida


## Preparing restaraunts dataset in Noida sector62

In [26]:
Restaraunts = dataframe_filtered[['name','id']]
num_rest = Restaraunts.shape[0]

Restaraunts

,name,id
0,Dev's Restaurant & Bar,52978e9811d2ca6acc6743b3
1,shakuntalam sweets namkeen & restaurant,521ccaea11d2a569f484368c
2,The Yellow Chilli Restaurant,4e9710167ee65b1674cd21e8
3,Khidmat Restaurant,52dfa8f211d254acf595bd1c
4,Bonanza Restaurant Vaishali sec 4,4ece290df5b9832ad7a6d184
5,1 The Restaurant,56d44a12cd10b9cf62f236ff
6,FEZ Restaurant,4b644892f964a52041a82ae3
7,Galords Restaurant,5297919411d2ca6acc682ff0
8,Green Leaf Restaurant,4e7dbf44f9f46ece9c8e489c
9,Haveli Restaurant,4e6a4cce52b1c1205fbf350b


## Preparing Users dataset 

__Note__ : Due to unavailable rated restaraunt users dataset, I am considering 10 random users and their ratings are also considered random.` Objective of my Project is to bulid recommendor sytem to users usiing different MachineLearning and DeepLearning models`. So, in this case users data is considered randomly

In [27]:
user_names = ['John','Robert','William','Charles','David','James','Richard','Johann','George','Paul']
users = pd.DataFrame(columns = user_names)
num_users = users.shape[1]

users

,John,Robert,William,Charles,David,James,Richard,Johann,George,Paul


In [56]:
user_rated = dict();
for i in range(num_users):
    user_rated[user_names[i]] = list(np.random.choice([0, 1], size=(num_rest,), p=[20/30, 10/30]))

user_rated = pd.DataFrame(user_rated)
user_ratings = user_rated.copy()


user_rated

,Charles,David,George,James,Johann,John,Paul,Richard,Robert,William
0,1,0,0,0,0,0,0,1,0,0
1,0,1,1,0,0,0,0,1,0,1
2,0,0,0,1,1,0,0,0,0,0
3,1,0,1,0,0,0,0,0,1,0
4,1,0,0,1,1,0,1,0,0,1
5,0,0,0,0,0,0,0,1,1,1
6,0,1,0,0,1,0,1,1,0,0
7,0,0,0,0,1,0,0,0,0,1
8,0,0,0,1,1,0,0,0,0,0
9,1,0,1,0,0,0,0,1,0,0


In [57]:
from random import randint

for i in range(num_rest):
    list1 = user_rated.iloc[i]
    list2 = list([randint(0, 5) for p in range(0, 10)])
    user_ratings.iloc[i] = list1*list2
    
user_ratings

,Charles,David,George,James,Johann,John,Paul,Richard,Robert,William
0,1,0,0,0,0,0,0,1,0,0
1,0,5,5,0,0,0,0,3,0,1
2,0,0,0,5,1,0,0,0,0,0
3,2,0,0,0,0,0,0,0,4,0
4,2,0,0,4,2,0,5,0,0,1
5,0,0,0,0,0,0,0,0,3,1
6,0,2,0,0,4,0,3,1,0,0
7,0,0,0,0,5,0,0,0,0,2
8,0,0,0,3,0,0,0,0,0,0
9,4,0,0,0,0,0,0,3,0,0


## Final Datset Preparation

In [58]:
user_ratings['venues'] = Restaraunts['name']
df = user_ratings.set_index('venues')

df

,Charles,David,George,James,Johann,John,Paul,Richard,Robert,William
venues,,,,,,,,,,
Dev's Restaurant & Bar,1,0,0,0,0,0,0,1,0,0
shakuntalam sweets namkeen & restaurant,0,5,5,0,0,0,0,3,0,1
The Yellow Chilli Restaurant,0,0,0,5,1,0,0,0,0,0
Khidmat Restaurant,2,0,0,0,0,0,0,0,4,0
Bonanza Restaurant Vaishali sec 4,2,0,0,4,2,0,5,0,0,1
1 The Restaurant,0,0,0,0,0,0,0,0,3,1
FEZ Restaurant,0,2,0,0,4,0,3,1,0,0
Galords Restaurant,0,0,0,0,5,0,0,0,0,2
Green Leaf Restaurant,0,0,0,3,0,0,0,0,0,0


In [59]:
Y = df.values
R = user_rated.values

Y

array([[1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 5, 5, 0, 0, 0, 0, 3, 0, 1],
       [0, 0, 0, 5, 1, 0, 0, 0, 0, 0],
       [2, 0, 0, 0, 0, 0, 0, 0, 4, 0],
       [2, 0, 0, 4, 2, 0, 5, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 3, 1],
       [0, 2, 0, 0, 4, 0, 3, 1, 0, 0],
       [0, 0, 0, 0, 5, 0, 0, 0, 0, 2],
       [0, 0, 0, 3, 0, 0, 0, 0, 0, 0],
       [4, 0, 0, 0, 0, 0, 0, 3, 0, 0],
       [4, 0, 0, 0, 0, 0, 0, 0, 0, 3],
       [0, 1, 0, 0, 1, 0, 0, 0, 4, 0],
       [0, 4, 0, 0, 0, 0, 0, 0, 0, 1],
       [2, 2, 2, 0, 0, 0, 0, 0, 0, 5],
       [1, 0, 2, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 5, 0, 0, 4],
       [4, 1, 4, 0, 2, 0, 1, 0, 3, 0],
       [0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 5, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 2, 0, 0, 0, 0, 0, 5],
       [0, 0, 0, 0, 1, 3, 3, 0, 0, 0],
       [0, 0, 0, 1, 2, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 3, 0, 0, 0, 0, 0, 0],
       [5, 0, 3, 0, 0, 0, 0, 0, 0, 1],
       [5, 0, 0, 5, 0, 0,

## Collabarative filtering Technique With Customised number of Features 

#### Here in this method, I considered 3`variable` features for recommendor system

In [60]:
### Sample values   
   
# num_rest = 5
# num_features=3
# num_users = 4
   

X_sample= np.array([[1.048686,  -0.400232,  1.194119],
           [0.780851,  -0.385626,   0.521198],
           [0.641509,  -0.547854,  -0.083796],
           [0.453618,  -0.800218,   0.680481],
           [0.937538,   0.106090,   0.361953]])

Theta_sample =np.array([[0.28544,  -1.68427,  0.26294],
               [0.50501,  -0.45465,   0.31746],
               [-0.43192,  -0.47880,  0.84671],
               [0.72860,  -0.27189,   0.32684]])

Y_sample =np.array([[5,   4,   0,   0],
           [3,   0,   0,   0],
           [4,   0,   0,   0],
           [3,   0,   0,   0],
           [3,   0,   0,   0]])

R_sample =  np.array([[1,  1,  0,  0],
              [1,  0,  0,  0],
              [1,  0,  0,  0],
              [1,  0,  0,  0],
              [1,  0,  0,  0]])


# cost function and gradient function


##### def cofiCostFunc(params, Y, R, num_users, num_rest, num_features, lamda):
    X = np.reshape(params[0:num_rest*num_features], (num_rest, num_features))
    Theta = np.reshape(params[num_rest*num_features:], (num_users, num_features))
    error = 0
    pred_values = np.around(X@Theta.T,2)
    
    pred_values_rated = np.multiply(pred_values, R)
    Y_rated = np.multiply(Y,R)
    difference = pred_values - Y
    difference_rated = np.multiply(difference,R)
    error = sum(sum(np.multiply(difference_rated,difference_rated)))
    
    J = (1/2)*(error)
    J = J + (   (lamda/2)*(  sum(sum(np.multiply(Theta,Theta)))  +  sum(sum(np.multiply(X,X)))  )    )
    
    X_grad = np.zeros(X.shape);
    Theta_grad = np.zeros(Theta.shape); 
    
    
    
    for i in range(num_rest):
        idx = np.where(R[i]==1)[0]
        Thetatemp = Theta[idx]
        Ytemp = Y[i][idx]
        X_grad[i] = ( ((X[i]@np.transpose(Thetatemp))-Ytemp)@Thetatemp  + lamda*X[i]   )
        
        
        
    for i in range(num_users):
        idx = np.where(R[i]==1)[0]
        Xtemp = X[idx]
        Ytemp = Y[idx,[i]]
        Theta_grad[i] = (  ((Theta[i]@np.transpose(Xtemp))-np.transpose(Ytemp))@Xtemp   + lamda*Theta[i]   )
        
        
    grad = np.reshape(np.concatenate((X_grad,Theta_grad), axis=0),(1,(num_rest+num_users)*num_features))[0]
    
    return [J, grad]

 


### STEP - 1 : Defining cost function

#### Defining the Cost function and Gradient functions

In [61]:
# normalising ratings inorder to overcome new user problem in recommnedor systems

def normalizeRatings(Y, R):
    m, n = Y.shape
    Ymean = np.zeros([m, 1]);
    Ynorm = np.zeros(Y.shape);
    for i in range(m):
        idx = np.where(R[i]==1)[0]
        Ymean[i] = np.mean(Y[i][idx])
        Ynorm[i,[idx]] = Y[i,[idx]] - Ymean[i];
    
    return Ynorm,Ymean



def cofiCostFunc(params, *args):
    X = np.reshape(params[0:num_rest*num_features], (num_rest, num_features))
    Theta = np.reshape(params[num_rest*num_features:], (num_users, num_features))
    error = 0
    pred_values = np.around(X@Theta.T,2)
    
    pred_values_rated = np.multiply(pred_values, R)
    Y_rated = np.multiply(Y,R)
    difference = pred_values - Y
    difference_rated = np.multiply(difference,R)
    error = sum(sum(np.multiply(difference_rated,difference_rated)))
    
    J = (1/2)*(error)
    J = J + (   (lamda/2)*( np.sum(np.square(Theta))  +  np.sum(np.square(X))  )    )
    
    return J



def grad_fun(params, *args):
    X = np.reshape(params[0:num_rest*num_features], (num_rest, num_features))
    Theta = np.reshape(params[num_rest*num_features:], (num_users, num_features))
    
    X_grad = np.zeros(X.shape);
    Theta_grad = np.zeros(Theta.shape); 
    
    for i in range(num_rest):
        idx = np.where(R[i]==1)[0]
        Thetatemp = Theta[idx]
        Ytemp = Y[i][idx]
        X_grad[i] = ( ((X[i]@np.transpose(Thetatemp))-Ytemp)@Thetatemp  + lamda*X[i]   )
        
    for i in range(num_users):
        idx = np.where(R[i]==1)[0]
        Xtemp = X[idx]
        Ytemp = Y[idx,[i]]
        Theta_grad[i] = (  ((Theta[i]@np.transpose(Xtemp))-np.transpose(Ytemp))@Xtemp   + lamda*Theta[i]   )
        
        
    grad = np.reshape(np.concatenate((X_grad,Theta_grad), axis=0),(1,(num_rest+num_users)*num_features))[0]
    
    return grad





def save_step(k):
    global steps
    steps.append(k)

steps = []

In [62]:
ar = [[1,2,3],
      [3,2,1]]
np.sum(np.square(ar))

28

### STEP-2 : setting initial parameters

#### Setting Intital weights and required parameters

In [63]:
# use values
num_users = Y.shape[1]
num_rest = Y.shape[0]
num_features = 5;


# Set Initial Parameters (Theta, X)
from random import *
from scipy import special, optimize
# from optimize import fmin_cg

X = np.zeros((num_rest, num_features))
Theta = np.zeros((num_users, num_features))

for i in range(num_rest):
    for j in range(num_features):
        X[i][j] = round(uniform(-1, 1),2)
        
        
for i in range(num_users):
    for j in range(num_features):
        Theta[i][j] = round(uniform(-1, 1),2)


### STEP - 3 : Normalization and Optimization

#### Normalize the ratings --- To overcome new user prediction problem

In [64]:
normalizeRatings(Y, R)

(array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  1.5       ,  1.5       ,  0.        ,  0.        ,
          0.        ,  0.        , -0.5       ,  0.        , -2.5       ],
        [ 0.        ,  0.        ,  0.        ,  2.        , -2.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        , -2.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  2.        ,  0.        ],
        [-0.8       ,  0.        ,  0.        ,  1.2       , -0.8       ,
          0.        ,  2.2       ,  0.        ,  0.        , -1.8       ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        , -1.33333333,  1.66666667, -0.33333333],
        [ 0.        , -0.5       ,  0.        ,  0.        ,  1.5       ,
          0.        ,  0.5      

In [65]:
# normalising the ratings
Ynorm, Ymean = normalizeRatings(Y, R)

#### Using fmin_cg function for OPTIMIZATION problem

In [66]:
initial_parameters = np.reshape(np.concatenate((X,Theta), axis=0),(1,(num_rest+num_users)*num_features))[0]


# Set Regularization
lamda = 10

args = (Y, R, num_users, num_rest, num_features, lamda)
theta = optimize.fmin_cg ( cofiCostFunc, initial_parameters, fprime=grad_fun ,args=args)


         Current function value: 407.085979
         Iterations: 5
         Function evaluations: 62
         Gradient evaluations: 56


In [67]:
Ymean

array([[ 1.        ],
       [ 3.5       ],
       [ 3.        ],
       [ 2.        ],
       [ 2.8       ],
       [ 1.33333333],
       [ 2.5       ],
       [ 3.5       ],
       [ 1.5       ],
       [ 2.33333333],
       [ 2.33333333],
       [ 1.5       ],
       [ 1.66666667],
       [ 2.75      ],
       [ 1.33333333],
       [ 4.5       ],
       [ 2.5       ],
       [ 1.        ],
       [ 2.5       ],
       [ 3.        ],
       [ 2.33333333],
       [ 0.75      ],
       [ 0.        ],
       [ 1.5       ],
       [ 2.25      ],
       [ 3.33333333],
       [ 1.5       ],
       [ 3.33333333],
       [ 3.        ],
       [ 3.25      ]])

### STEP - 4 : Predicting User Ratings

In [68]:
users_intrst_pred = np.reshape(theta[0:num_rest*num_features], (num_rest, num_features))
movie_features_pred = np.reshape(theta[num_rest*num_features:], (num_users, num_features))

predictions = (users_intrst_pred@movie_features_pred.T)+Ymean

predictions = np.around(predictions,1)
predictions

array([[ 1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ],
       [ 3.5,  3.5,  3.5,  3.5,  3.5,  3.5,  3.5,  3.5,  3.5,  3.5],
       [ 3. ,  3. ,  3. ,  3. ,  3. ,  3. ,  3. ,  3. ,  3. ,  3. ],
       [ 2. ,  2. ,  2. ,  2. ,  2. ,  2. ,  2. ,  2. ,  2. ,  2. ],
       [ 2.8,  2.8,  2.8,  2.8,  2.8,  2.8,  2.8,  2.8,  2.8,  2.8],
       [ 1.3,  1.3,  1.3,  1.3,  1.3,  1.3,  1.3,  1.3,  1.3,  1.3],
       [ 2.5,  2.5,  2.5,  2.5,  2.5,  2.5,  2.5,  2.5,  2.5,  2.5],
       [ 3.5,  3.5,  3.5,  3.5,  3.5,  3.5,  3.5,  3.5,  3.5,  3.5],
       [ 1.5,  1.5,  1.5,  1.5,  1.5,  1.5,  1.5,  1.5,  1.5,  1.5],
       [ 2.3,  2.3,  2.3,  2.3,  2.3,  2.3,  2.3,  2.3,  2.3,  2.3],
       [ 2.3,  2.3,  2.3,  2.3,  2.3,  2.3,  2.3,  2.3,  2.3,  2.3],
       [ 1.5,  1.5,  1.5,  1.5,  1.5,  1.5,  1.5,  1.5,  1.5,  1.5],
       [ 1.7,  1.7,  1.7,  1.7,  1.7,  1.7,  1.7,  1.7,  1.7,  1.7],
       [ 2.8,  2.7,  2.8,  2.8,  2.8,  2.7,  2.8,  2.8,  2.7,  2.8],
       [ 1.3,  1.3,  1.3,  1.3,  1

In [69]:
idx = np.where(R[:,0]==0)[0]

user_not_rated_move_rating = np.zeros([30,1])
user_not_rated_move_rating[idx,0] = predictions[idx,0]
user_not_rated_move_rating = user_not_rated_move_rating.T 

top_5_list = np.argsort(user_not_rated_move_rating)[0][::-1]

venue_names = df.index


### STEP - 5: Recommendations

In [70]:


print('Top 5 recommended Restaraunts to Jhon with predicted ratings: ',end='\n\n')
for i in range(5):
    venue_name = venue_names[top_5_list[i]]
    user_rating = user_not_rated_move_rating[0][top_5_list[i]]
    
    print("{0:50} {1}".format(venue_name, user_rating))


Top 5 recommended Restaraunts to Jhon with predicted ratings: 

mehfill restaurant                                 4.5
Galords Restaurant                                 3.5
shakuntalam sweets namkeen & restaurant            3.5
Anand Restaurant                                   3.3
The Yellow Chilli Restaurant                       3.0


## Decomposing Rating Matrix Using SVD

### STEP - 2: setting initial parameters

In [71]:
P, D, Q = np.linalg.svd(Y, full_matrices=False)

# parameters
num_features = Q.shape[1]
num_users = Y.shape[1]
num_rest = Y.shape[0]
Theta = np.diag(D)@Q
X=P



### STEP - 3 : Normalization and Optimization

In [72]:
# normalising the ratings
Ynorm, Ymean = normalizeRatings(Y, R)

#### Using fmin_cg function for OPTIMIZATION problem

In [73]:
initial_parameters = np.reshape(np.concatenate((X,Theta), axis=0),(1,(num_rest+num_users)*num_features))[0]


# Set Regularization
lamda = 10

args = (Y, R, num_users, num_rest, num_features, lamda)
theta = optimize.fmin_cg ( cofiCostFunc, initial_parameters, fprime=grad_fun ,args=args)


         Current function value: 407.008211
         Iterations: 8
         Function evaluations: 154
         Gradient evaluations: 145


In [74]:
Ymean

array([[ 1.        ],
       [ 3.5       ],
       [ 3.        ],
       [ 2.        ],
       [ 2.8       ],
       [ 1.33333333],
       [ 2.5       ],
       [ 3.5       ],
       [ 1.5       ],
       [ 2.33333333],
       [ 2.33333333],
       [ 1.5       ],
       [ 1.66666667],
       [ 2.75      ],
       [ 1.33333333],
       [ 4.5       ],
       [ 2.5       ],
       [ 1.        ],
       [ 2.5       ],
       [ 3.        ],
       [ 2.33333333],
       [ 0.75      ],
       [ 0.        ],
       [ 1.5       ],
       [ 2.25      ],
       [ 3.33333333],
       [ 1.5       ],
       [ 3.33333333],
       [ 3.        ],
       [ 3.25      ]])

### STEP - 4 : Predicting User Ratings

In [75]:
users_intrst_pred = np.reshape(theta[0:num_rest*num_features], (num_rest, num_features))
movie_features_pred = np.reshape(theta[num_rest*num_features:], (num_users, num_features))

predictions = (users_intrst_pred@movie_features_pred.T)+Ymean

predictions = np.around(predictions,1)
predictions

array([[ 1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ],
       [ 3.5,  3.5,  3.5,  3.5,  3.5,  3.5,  3.5,  3.5,  3.5,  3.5],
       [ 3. ,  3. ,  3. ,  3. ,  3. ,  3. ,  3. ,  3. ,  3. ,  3. ],
       [ 2. ,  2. ,  2. ,  2. ,  2. ,  2. ,  2. ,  2. ,  2. ,  2. ],
       [ 2.8,  2.8,  2.8,  2.8,  2.8,  2.8,  2.8,  2.8,  2.8,  2.8],
       [ 1.3,  1.3,  1.3,  1.3,  1.3,  1.3,  1.3,  1.3,  1.3,  1.3],
       [ 2.5,  2.5,  2.5,  2.5,  2.5,  2.5,  2.5,  2.5,  2.5,  2.5],
       [ 3.5,  3.5,  3.5,  3.5,  3.5,  3.5,  3.5,  3.5,  3.5,  3.5],
       [ 1.5,  1.5,  1.5,  1.5,  1.5,  1.5,  1.5,  1.5,  1.5,  1.5],
       [ 2.3,  2.3,  2.3,  2.3,  2.3,  2.3,  2.3,  2.3,  2.3,  2.3],
       [ 2.3,  2.3,  2.3,  2.3,  2.3,  2.3,  2.3,  2.3,  2.3,  2.3],
       [ 1.5,  1.5,  1.5,  1.5,  1.5,  1.5,  1.5,  1.5,  1.5,  1.5],
       [ 1.7,  1.7,  1.7,  1.7,  1.7,  1.7,  1.7,  1.7,  1.7,  1.7],
       [ 2.8,  2.8,  2.8,  2.8,  2.8,  2.8,  2.8,  2.7,  2.7,  2.8],
       [ 1.3,  1.3,  1.3,  1.3,  1

In [76]:
idx = np.where(R[:,0]==0)[0]

user_not_rated_move_rating = np.zeros([30,1])
user_not_rated_move_rating[idx,0] = predictions[idx,0]
user_not_rated_move_rating = user_not_rated_move_rating.T 

top_5_list = np.argsort(user_not_rated_move_rating)[0][::-1]

venue_names = df.index


### STEP - 5: Recommendations

In [77]:


print('Top 5 recommended Restaraunts to Jhon with predicted ratings: ',end='\n\n')
for i in range(5):
    venue_name = venue_names[top_5_list[i]]
    user_rating = user_not_rated_move_rating[0][top_5_list[i]]
    
    print("{0:50} {1}".format(venue_name, user_rating))


Top 5 recommended Restaraunts to Jhon with predicted ratings: 

mehfill restaurant                                 4.5
Galords Restaurant                                 3.5
shakuntalam sweets namkeen & restaurant            3.5
Anand Restaurant                                   3.3
The Yellow Chilli Restaurant                       3.0


In [78]:
num_features

10